# Сбор данных с Web-scraping и API для социально-научных исследований
---
## Семинары 13-14. Введение в Selenium. Работа с открытыми судебными данными. Менеджмент файлов на компьютере с помощью os.

---
*ФСН, ОП "Политология", 2023-2024 гг.*

Лика Капустина,

lkapustina@hse.ru

**План занятия:**
1. [1. Selenium: введение и использование web-driver](#part1)
2. [2. Selenium: работа с элементами страницы](#part2)
3. [3. Selenium: обработка html страницы (привет, BeautifulSoup)](#part3)
4. [4. Selenium: явные и неявные ожидания и обработка исключений](#part4)
5. [5. os: менеджмент файлов на вашем компьютере](#part5)
---

**Основные ссылки:**
- [Документация Selenium на Python](https://selenium-python.readthedocs.io);
- [Неофициальный перевод документации Selenium на Python на русский](https://habr.com/ru/articles/248559/);
- [Документация BeautifulSoup на русском](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html);
- [Документация os](https://docs.python.org/3/library/os.html);
- [Распространенные exceptions в Selenium](https://www.selenium.dev/selenium/docs/api/py/common/selenium.common.exceptions.html).

In [3]:
# импорт основных библиотек
import pandas as pd # работа с таблицами
import numpy as np  # математические операции и пр.
import tqdm         # для дальнейшего импорта прогресс-бара
import time         # для обычных пауз

## 1. Selenium: введение и использование web-driver<a name='part1'></a>

**Что такое Selenium?**
    
Selenium – программная библиотека для управления браузерами. Появилась в 2011 году и используется преимущественно для автоматизированного тестирования сайтов, но получила распространение и как инструмент, с помощью которого можно взаимодействовать с Javascript на сайтах и имитировать действия реального пользователя. 
    
**Ниже я продемонстрирую одну из причин, по которой вам иногда необходимо использовать Selenium**. 

Некоторые сайты не отдают html страниц в ответ на запросы ботов:

In [51]:
import requests # импортируем requests для отправки запросов
from bs4 import BeautifulSoup # импортируем BeautifulSoup

# получим html-разметку со страницы поиска Московского Городского Суда
html = requests.get('https://mos-gorsud.ru/search') # сайт московского городского суда
soup = BeautifulSoup(html.text) # получаем суп
print(soup.prettify()) # что тут? Да, тут пусто

Или в качестве `response` возвращают объекты с неполным содержанием. Например, сейчас мы попытаемся отправить запрос к странице [Высшей школы экономики в ВКонтакте](https://vk.com/hse). На этой странице содержится информация о группе Вышки в ВК, а также посты сообщества.

In [70]:
html = requests.get('https://vk.com/hse') # отправляем запрос
soup = BeautifulSoup(html.text) # обрабатываем html
#print(soup.prettify())

Зачастую единственное решение в таких случаях – либо пользоваться API сайта (если оно существует), либо использовать Selenium для имитации браузера и действий реального пользователя.

### Использование Selenium webdriver

**Что такое `webdriver`?** Webdriver – это основная сущность в Selenium, которая отвечает за управление браузером. 

Сейчас мы воспользуемся `WebdriverManager` – это класс, который поможет нам не мучаться с установкой WebDriver под каждую новую версию Google Chrome.

**Альтернатива**. Альтернативный способ установки webdriver для автоматизированной работы с браузером с помощью Selenium представляет из себя скачивание [ChromeDriver](https://chromedriver.chromium.org) под определенную версию Chrome. Это выглядело так: вы скачиваете Chromedriver под определенную версию своего браузера Chrome и каждый раз после выхода новой версии Google Chrome вы заново скачиваете подходящую версию ChromeDriver. С выхода версии 115 Google Chrome процесс скачивания подходящей версии стал [сложнее](https://www.browserstack.com/guide/run-selenium-tests-using-selenium-chromedriver). **Поэтому я советую вам пользоваться установкой Chromedriver через `ChromeDriverManager()`.**

In [2]:
#!pip install selenium # установка selenium
#!pip install webdriver-manager # установка пакета webdriver_manager

**`webdriver.Chrome()`, `ChromeDriverManager()`, `wb.get()`**

С помощью кода ниже мы cоздадим `selenium.webdriver.chrome.webdriver.WebDriver`, откроем определенную страницу, а по прошествии 5 секунд она будет закрыта. Такая запись с помощью `with` позволяет создать объект webdriver в первой строчке, а далее прописать код, который будет использован до автоматического прекращения работы webdriver (по окончанию последней строчки кода):

In [8]:
from selenium import webdriver # импорт драйвера
from selenium.webdriver.chrome.service import Service # импортируем сущность Service
from webdriver_manager.chrome import ChromeDriverManager # импорт драйвера для Google Chrome


with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as wb:
    wb.get("https://www.hse.ru/edu/courses/875214615") # открываем определенную страницу
    time.sleep(5) # открываем страницу на 5 секунд, через 5 секунд она закроется

**Есть альтернатива**

Используйте строчку кода ниже, чтобы открыть окно Google Chrome с помощью webdriver и присвоить имя `wb` вашему `selenium.webdriver.chrome.webdriver.WebDriver`.

In [182]:
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # делается это так

In [18]:
wb # можем исследовать наш объект Webdriver

<selenium.webdriver.chrome.webdriver.WebDriver (session="edfdc034014c87ad28421c29b4fa2700")>

In [21]:
type(wb) # и посмотреть на его тип

selenium.webdriver.chrome.webdriver.WebDriver

**`selenium.webdriver.chrome.options`**

При использовании `Selenium` вы можете использовать большое количество разных возможностей: например, открыть окно браузера в полный экран. Для этого нужно поработать с классом `Options()`:

In [72]:
from selenium.webdriver.chrome.options import Options 

options = Options() # создаем сущность Options
options.add_argument("start-maximized") # уточняем, что хотим чтобы при открытии окно было открыто максимально широко
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # создаем вебдрайвер
wb.get("https://www.hse.ru/edu/courses/875214615") # открываем ссылку

**Ручное взаимодействие с открытой страницей браузера и `wb.current_url`** 

С помощью метода `wb.current_url` вы можете получить ссылку на конкретную страницу. Зачем это нужно? В некоторых проектах вы можете переключаться между страницами (например, страницами новостей, профилями в социальных сетях) с помощью кнопок и других элементов (то есть, у вас не будет изначального списка ссылок, по которому вы будете идти). Чтобы сохранить в датафрейм информацию по данной конкретной странице, вам нужно будет использовать метод `wb.current_url`.

**Другая особенность Selenium – вы можете переключаться между разными страницами в открытом вами окне и браузер будет запоминать ваши действия**. Например:
* Вы можете открыть страницу социальной сети, залогиниться там вручную (не посылая значения в поля), а дальше использовать скрипт, написанный с использованием Selenium для выполнения автоматических действий. 
* Вы можете открыть страницу социальной сети, залогиниться там вручную (не посылая значения в поля), вручную ввести код подтверждения который пришел вам на телефон, а дальше использовать скрипт, написанный с использованием Selenium для выполнения автоматических действий.
* Вы можете открыть страницу и нажать на определенную кнопку настроек (например, *переключиться на светлую тему* или *скрыть уведомления*), и при работе с открытым в рамках данной сессии окном вебдрайвер запомнит настройки на этом сайте.

Продемонстрируем на примере, что браузер проследит за вашими действиями:

In [73]:
time.sleep(3)
print(wb.current_url) # это наша нынешняя ссылка

'https://www.hse.ru/edu/courses/875214615'

In [74]:
# давайте немного походим по сайту

In [75]:
# и теперь напечатаем ссылку страницы, которую мы только что открыли
time.sleep(3)
print(wb.current_url)

https://www.hse.ru/


**`wb.close()` и `wb.quit()`**

Вы также можете закрыть открытое с помощью вебдрайвера окно. Это можно сделать с помощью метода `wb.close()` или `wb.quit()`.

* `wb.close()` - закрывает браузер, который сейчас открыт;

* `wb.quit()` - закрывает все окна и уничтожает сущность Webdriver. На практике вы не почувствуете разницы.

In [183]:
time.sleep(5) # сперва подождем пять секунд
wb.close()    # теперь закроем открытое окно

In [181]:
time.sleep(3) # подождем три секунды
wb.quit()     # другой способ

Обратите внимание - когда вы закрываете окно, вы больше **не можете взаимодействовать с тем окном, которым пользовались раньше**. Вам придется вновь создавать webdriver (`selenium.webdriver.chrome.webdriver.WebDriver`) и взаимодействовать уже с новым окном.

In [16]:
wb.current_url # ошибка

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000100fef538 chromedriver + 4687160
1   chromedriver                        0x0000000100fe6d83 chromedriver + 4652419
2   chromedriver                        0x0000000100bd7dcb chromedriver + 396747
3   chromedriver                        0x0000000100c15ac7 chromedriver + 649927
4   chromedriver                        0x0000000100c467e4 chromedriver + 849892
5   chromedriver                        0x0000000100c412f2 chromedriver + 828146
6   chromedriver                        0x0000000100c40b6d chromedriver + 826221
7   chromedriver                        0x0000000100ba46bf chromedriver + 186047
8   chromedriver                        0x0000000100faf980 chromedriver + 4426112
9   chromedriver                        0x0000000100fb4c18 chromedriver + 4447256
10  chromedriver                        0x0000000100f93e81 chromedriver + 4312705
11  chromedriver                     

## 2. Selenium: работа с элементами страницы<a name='part2'></a>

### Selenium и работа с элементами страницы

**Другая причина по которой пользуются Selenium – это возможность взаимодействовать с элементами страницы: отправлять значения в формы, нажимать на кнопки и пр.** До некоторой информации невозможно добраться с помощью `requests` - например, если предварительно на сайте вам нужно нажать на кнопки вроде `Развернуть` и `Показать все`. С помощью Selenium вы можете взаимодействовать с этими самыми элементами.

**`wb.find_element(By.)`**

Что такое `.By`? `By` - класс, содержащий статические методы для идентификации элементов.

В свою очередь, `webelement` – это сущность, представляющая собой абстракцию над каким-то веб-элементом (кнопки, ссылки, поля ввода).

Вы можете использовать разные методы `.By` чтобы находить конкретные элементы:
<p></p>
<center>
    <table>
        <tr>
            <th><center>By.</center></th>
            <th><center>Описание</center></th>
        <tr><td>ID</td>
            <td>id конкретного элемента html</td></tr>
        <tr><td>TAG_NAME</td>
        <td>Название тега конкретного элемента html</td></tr>
        <tr><td>CLASS_NAME</td>
        <td>Название класса конкретного элемента html</td></tr>
        <tr><td>СSS_SELECTOR</td>
        <td>Локатор, который включает в себя конкретные уникальные атрибуты элемента html</td></tr>
        <tr><td>XPATH</td>
        <td>Путь по дереву html к элементу</td></tr>
        <tr><td>NAME</td>
            <td>Значение атрибута <code>name</code> конкретного элемента html</td></tr>
        <tr><td>PARTIAL_LINK_TEXT</td>
        <td>Возвращает элемент html, который содержит этот текст</td></tr>
        <tr><td>LINK_TEXT</td>
        <td>Возвращает элемент html, который содержит в точности этот текст</td></tr>
    </table>
</center>

**`wb.find_element(By.)` и `wb.find_elements(By.)`**: точно также, как при работе с объектами `bs4.BeautifulSoup` у вас есть возможность пользоваться двумя методами для поиска по элементам:
* Методом, возвращающим первый элемент, удовлетворяющий условию – `wb.find_element(By., ..)`
* Методом, возвращающим все элементы, удовлетворяющие условию – `wb.find_elements(By.)`.

Использование метода `wb.find_elements(By.)` удобно, когда вам нужно найти сразу несколько элементов, с которыми вы потом хотите взаимодействовать: например, если вы хотите найти несколько кнопок с одинаковым классом чтобы в дальнейшем в цикле прокликать их. 

Полная документация по поиску элементов с помощью Selenium [доступна по ссылке](https://selenium-python.readthedocs.io/locating-elements.html).

---

Давайте попробуем поискать элементы на странице поиска по судебным делам на сайте **[Московского городского суда](https://mos-gorsud.ru/search)**. Сайт Московского городского суда позволяет получить информацию о делах в самом Московском городском суде (суде субъекта федерации), так и о делах, которые рассматриваются в московских районных судах (наиболее распространенный уровень судов в России кроме мировых судей). Сперва откроем страницу:

In [185]:
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # создаем сущность вебдрайвера
url = 'https://mos-gorsud.ru/search' # сохраняем ссылку в переменную
wb.get(url) # открываем эту самую ссылку

Обратим внимание на элемент **<u>Нажмите, чтобы развернуть таблицу</u>**. 

Кликнем на него правой кнопкой мыши, нажмем `Просмотреть код`. 

Как мы видим, в html-разметке страницы этому элементу соответствует следующая строчка:

```<a href="#" class="js-table-modal-trigger">Нажмите, чтобы развернуть таблицу</a>```

**Как можно найти этот конкретный элемент на странице?** В случае с элементами, которые содержат в себе ссылки, можно использовать такие способы поиска:

In [25]:
from selenium.webdriver.common.by import By # импортируем класс .By
wb.find_element(By.LINK_TEXT, 'Нажмите, чтобы развернуть таблицу') # ищем элемент с ссылкой по конкретному значению

<selenium.webdriver.remote.webelement.WebElement (session="e19e46afc29723154422089c818a62a4", element="C4A9C2AC6FC7A176A280E36D8E8FDE9F_element_4717")>

In [28]:
wb.find_element(By.PARTIAL_LINK_TEXT, 'Нажмите') # ищем элемент по частичному тексту ссылки

<selenium.webdriver.remote.webelement.WebElement (session="e19e46afc29723154422089c818a62a4", element="C4A9C2AC6FC7A176A280E36D8E8FDE9F_element_4717")>

Можно искать элемент по названию класса:

In [29]:
wb.find_element(By.CLASS_NAME, 'js-table-modal-trigger') # нашли элемент по названию class

<selenium.webdriver.remote.webelement.WebElement (session="e19e46afc29723154422089c818a62a4", element="C4A9C2AC6FC7A176A280E36D8E8FDE9F_element_4717")>

Но есть также и другие способы. Например, поиск по css-selector'у, xpath и пр. 

**Как его выполнить?**

Для этого сперва нужно будет обратиться к конкретному элементу, нажать на него правой кнопкой мыши, навести курсор на `Копировать`/`Copy`, и далее в выпадающем окне выбрать одно из значений. Например, при выборе `Копировать selector`/`Copy css-selector` вы получите css-selector: так вы сможете найти конкретный элемент, используя синтаксис css-selector'ов.

In [30]:
wb.find_element(By.CSS_SELECTOR, '#content > div > div.resultsearch_text > div > a') # скопировали css-selector

<selenium.webdriver.remote.webelement.WebElement (session="e19e46afc29723154422089c818a62a4", element="C4A9C2AC6FC7A176A280E36D8E8FDE9F_element_4717")>

Другой способ - скопировать **xpath**. xpath – это путь к элементу html по дереву html документа. Чтобы получить xpath элемента, вам нужно навести на конкретный элемент, кликнуть правой кнопкой мыши, нажать `Копировать xpath` или `Копировать полную строку xpath`.

In [31]:
wb.find_element(By.XPATH, '//*[@id="content"]/div/div[1]/div/a') # просто xpath

<selenium.webdriver.remote.webelement.WebElement (session="e19e46afc29723154422089c818a62a4", element="C4A9C2AC6FC7A176A280E36D8E8FDE9F_element_4717")>

In [32]:
wb.find_element(By.XPATH, '/html/body/div[2]/main/section/div[2]/div/div[1]/div/a') # полный xpath

<selenium.webdriver.remote.webelement.WebElement (session="e19e46afc29723154422089c818a62a4", element="C4A9C2AC6FC7A176A280E36D8E8FDE9F_element_4717")>

**`.click()`**

Зачем мы ищем конкретные элемены? Правильно, чтобы с ними провзаимодействовать. Со многими элементами можно провзаимодействовать, просто кликнув на них. За это отвечает метод `.click()`. 

In [34]:
time.sleep(3) # делаем паузу
button_open_table = wb.find_element(By.CSS_SELECTOR, '#content > div > div.resultsearch_text > div > a') # элемент
button_open_table.click() # кликаем на конкретный элемент
time.sleep(3) # делаем паузу

**Поздравляю! Теперь вы научились находить элементы и кликать на них.**

<p></p>
<center><b><font size=4>Задача №1. Поиск элементов страницы. </font></b></center>

**На предыдущем шаге нам удалось открыть таблицу. Теперь напишите код который закроет эту таблицу, и используйте конструкцию `try-except`, чтобы этот код выглядел следующим образом:**

* Попытайтесь найти элемент по названию класса и кликнуть на него;
    * Если не получилось – попытайтесь найти элемент по css_selector и кликнуть на него;
        * Если не получилось - попытайтесь найти элемент по xpath и кликнуть на него;
            * Если не получилось - напечатайте строку `Увы, не получилось!` и кликнуть на него.
            
В каждом блоке кода после `except` добавьте паузу на 2 секунды.

In [ ]:
# YOUR CODE HERE

### Selenium и отправка значений в поля

Кроме того, что с помощью Selenium вы можете искать конкретные значения и кликать на них, вы также можете работать с **полями для ввода информации**

Сперва кликнем на кнопку `Расширенная форма`, чтобы иметь возможности провзаимодействовать с полями для расширенного поиска по судебным делам.

In [76]:
# короче отправляем всякие параметры в поля запроса
url = 'https://mos-gorsud.ru/search' # сохраняем ссылку в переменную
wb.get(url) # открываем эту самую ссылку

In [78]:
time.sleep(3) # подождем три секунды
extended_search = wb.find_element(By.CLASS_NAME, 'extended ') # class этого элемента - "extended "
extended_search.click() # кликнем на элемент

Теперь наша задача - найти интересующие нас поля. Давайте найдем поле `Статья` и скопируем `css-selector` элемента, который соответствует этому полю на сайте:

In [80]:
article_field = wb.find_element(By.CSS_SELECTOR, '#codex') 
print(article_field) # элемент найден

<selenium.webdriver.remote.webelement.WebElement (session="aea96f84067c7604c92f48b8a15a7a23", element="829575CBFFFD6CB51E6DCBE0C64C140B_element_36")>


**`.send_keys()`**

Теперь обратимся к методу `.send_keys()`. Этот метод работает так:
* В начале - название поля, в которое вы собираетесь отправить определенное значение;
* далее - название метода - `.send_keys()`;
* в скобках - значение, которое вы планируете отправить в определенную форму.

Давайте выполним поиск дел по статье **212 Уголовного кодекса Российской Федерации**: найдем дела о [массовых беспорядках](https://www.consultant.ru/document/cons_doc_LAW_10699/cdfbaa9aeaf8b47695af18e41433e4e3f5f4be5f/).

In [81]:
from selenium.webdriver.common.keys import Keys # посылка текста для ввода в формы (логина, пароля, поиска)

article = 'Ст. 212'
article_field.send_keys(article + '\n') # в конце добавляем \n чтобы сразу запустить поиск
time.sleep(5) # делаем паузу

**Поздравляю! Вы научились отправлять значения в поля**.

<p></p>
<center><b><font size=4>Задача №2. Работа с полями и элементами страницы</font></b></center>

**Используя расширенный поиск, вернитесь к главной странице, введите следующие значения в соответствующие поля и выполните поиск по сайту:**

* `Инстанция` – первая (подумайте, как можно выбрать это значение);
* `Год` – **2023**;
* В `Тексте` присутствует словосочетание **НИУ ВШЭ**.

**Покажите преподавательнице, что у вас открылось.**

In [98]:
url = 'https://mos-gorsud.ru/search'
wb.get(url)

# YOUR CODE HERE

## 3. Selenium: обработка html страницы<a name='part3'></a>

Теперь давайте обсудим, как вы можете переключаться между страницами. **Во-первых, вы можете найти кнопку и кликнуть на кнопку "следующая страница"**.

In [91]:
next_page_button = wb.find_element(By.CLASS_NAME, 'next') # кнопка "далее"
print(next_page_button)

<selenium.webdriver.remote.webelement.WebElement (session="aea96f84067c7604c92f48b8a15a7a23", element="5DFD9010371C55777FADF4F944495885_element_7181")>


После того, как элемент найден, осталось только на него кликнуть.

In [92]:
time.sleep(3) # делаем паузу
next_page_button.click() # нажимаем на кнопку

При желании вы в цикле могли бы пройтись по всем страницам и каждый раз для перехода на следующую страницу нажимать на эту кнопку.

**Но есть ли более оптимальные опции?** Да. Для этого давайте взглянем на ссылку: ```https://mos-gorsud.ru/search??documentText=НИУ+ВШЭ&instance=1&year=2023&formType=fullForm&page=2```. 

Интересно, что если просто кликнуть на адресное поле и скопировать значение, вы получите отличающуюся строку: ```https://mos-gorsud.ru/search?documentText=%D0%9D%D0%98%D0%A3+%D0%92%D0%A8%D0%AD&instance=1&year=2023&formType=fullForm&page=2```.

Если воспользоваться методом `.current_url`, он вернет то же самое: ```https://mos-gorsud.ru/search?documentText=%D0%9D%D0%98%D0%A3+%D0%92%D0%A8%D0%AD&instance=1&year=2023&formType=fullForm&page=2'```.

Давайте сразу сгенерируем ссылки, которые в дальнейшем могли бы обойти в цикле и проверим, работают ли они:

In [97]:
pages_with_cases = \
[f'https://mos-gorsud.ru/search?codex=Ст.+212&formType=fullForm&page={i}' \
 for i in range(1, 47+1)] # поскольку на сайте всего 47 страниц с ссылками на дела по статье Ст. 212 УК РФ

wb.get(pages_with_cases[0]) # откроем первую ссылку
time.sleep(3) # сделаем паузу
wb.get(pages_with_cases[-1]) # откроем последнюю ссылку

**`wb.page_source`**

**Перейдем к обсуждению работы с html страницы**. На этапе сбора данных работа с html страницы ничем не отличается от того, что мы с вами делали когда обсуждали `requests`.

Давайте соберем ссылки на все дела, которые выпали нам в поиске с параметрами `Первая инстанция`, `2023 год`, в тексте присутствует `НИУ ВШЭ`.

In [107]:
need_link = wb.current_url # текущая ссылка
print(need_link)

https://mos-gorsud.ru/search?formType=fullForm&courtAlias=&uid=&instance=1&processType=&letterNumber=&caseNumber=&participant=&codex=&judge=&publishingState=&documentType=&documentText=%D0%9D%D0%98%D0%A3+%D0%92%D0%A8%D0%AD&year=2023&caseDateFrom=&caseDateTo=&caseFinalDateFrom=&caseFinalDateTo=&caseLegalForceDateFrom=&caseLegalForceDateTo=&docsDateFrom=&docsDateTo=&documentStatus=


In [111]:
html = wb.page_source # сохраним html страницы с помощью .page_source
soup = BeautifulSoup(html) # сварим суп

In [113]:
#soup # посмотрим на него

Давайте создадим список, в который соберем все ссылки на дела со страницы поиска. Как выглядит элемент с ссылкой на дело?

```<a target="_blank" class="detailsLink" href="/rs/basmannyj/services/cases/civil/details/7da80261-cd6f-11ed-91bc-678868b51ab8?year=2023&amp;formType=fullForm">02-3282/2023</a>```

Давайте воспользуемся атрибутом `class` и его значением - `detailsLink`. Далее мы обращаемся к нашему объекту `soup` и пользуемся старым добрым методом `.find_all()`, чтобы найти все элементы, удовлетворяющие деталям поиска:

*Если вы забыли как пользоваться методами BeautifulSoup, посмотрите материалы семинаров 3-6 или обратитесь к [документации BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html)*.

In [122]:
links = [i.get('href') for i in soup.find_all('a', {'class': 'detailsLink'})]
links = [f'https://mos-gorsud.ru{part_link}' for part_link in links]
links = links[::2]

Откроем первую ссылку

In [123]:
wb.get(links[0]) # все работает!

<p></p>
<center><b><font size=4>Задача №3. Сбор данных со страницы</font></b></center>

**Шаг 1. Вспомните работу с `BeautifulSoup` и напишите функцию `get_info_about_case(link)`, которая принимает на вход ссылку на страницу дела, и возвращает словарь (`dict`) с информацией из таблички в начале страницы и ссылки:**

Например, для первого дела вы получите следующий результат:
```
{'Уникальный идентификатор дела': '77RS0018-02-2023-004059-43',
 'Номер дела ~ материала': '02-4668/2023                                        ∼                                     М-2315/2023',
 'Стороны': 'Истец: Борсов А.И.Ответчик: ФГАОУ ВО "Национальный исследовательский университет "Высшая школа экономики"',
 'Дата поступления': '24.03.2023',
 'Дата рассмотрения дела в первой инстанции': '20.06.2023',
 'Дата вступления решения в силу': '14.12.2023',
 'Cудья': 'Юдина И.В.',
 'Категория дела': '179 - О защите прав потребителей - из договоров в сфере торговли, услуг, выполнения работ',
 'Номер дела в суде вышестоящей инстанции': '33-48031/2023',
 'Текущее состояние': 'Удовлетворено, 20.06.2023',
 'Решение апелляции': 'Отменено, 14.12.2023',
 'link': 'https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm'}
```

In [126]:
# YOUR CODE HERE

**Шаг 2. Напишите функцию `get_cases_data(links)`, которая принимает на вход список с ссылками `links`, проходится по нему, собирает список из словарей с помощью функции `get_info_about_case()`, и возвращает `pandas.DataFrane` с информацией о судебных делах**

In [ ]:
# YOUR CODE HERE

In [170]:
data # Так должен выглядеть результат

,Уникальный идентификатор дела,Номер дела ~ материала,Стороны,Дата поступления,Дата рассмотрения дела в первой инстанции,Дата вступления решения в силу,Cудья,Категория дела,Номер дела в суде вышестоящей инстанции,Текущее состояние,Решение апелляции,link,Подсудимый
0,77RS0018-02-2023-004059-43,02-4668/2023 ...,"Истец: Борсов А.И.Ответчик: ФГАОУ ВО ""Национал...",24.03.2023,20.06.2023,14.12.2023,Юдина И.В.,179 - О защите прав потребителей - из договоро...,33-48031/2023,"Удовлетворено, 20.06.2023","Отменено, 14.12.2023",https://mos-gorsud.ru/rs/nikulinskij/services/...,NaN
1,77RS0031-02-2023-003539-41,02-4564/2023 ...,Истец: Дружинин А.А.Ответчик: ФГАОУ высшего об...,03.03.2023,11.05.2023,NaN,Наделяева Е.И.,44 - О восстановлении на государственной (муни...,33-4121/2024,"Удовлетворено, 11.05.2023",NaN,https://mos-gorsud.ru/rs/horoshevskij/services...,NaN
2,77RS0002-02-2022-020618-86,02-3282/2023 ...,"Истец: Черничкин А.С.Ответчик: ПАО ""Сбербанк Р...",12.10.2022,26.07.2023,NaN,Курносова О.А.,178 - О защите прав потребителей - из договоро...,NaN,Назначено судебное заседание на 14.03.2024 09:50,NaN,https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
3,77RS0033-02-2023-003981-64,02-2500/2023 ...,Истец: Солдатенко В.А.Ответчик: Федеральное Го...,03.03.2023,15.08.2023,NaN,Молодцова Е.В.,211- Прочие исковые дела,"33-40411/2023 , ...","Удовлетворено частично, 15.08.2023",NaN,https://mos-gorsud.ru/rs/chertanovskij/service...,NaN
4,77RS0002-02-2022-023218-46,02-1845/2023 ...,Истец: Андриянова Д.А.Ответчик: Федеральное го...,25.11.2022,10.04.2023,18.05.2023,Старовойтова К.Ю.,54 - Трудовые споры о взыскании невыплаченной ...,NaN,"Удовлетворено частично, 10.04.2023",NaN,https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
5,77RS0002-02-2022-016892-12,02-1124/2023 (02-55...,"Истец: Барабанщиков К.Ю.Ответчик: ФГАОУВО ""НИУ...",17.08.2022,07.02.2023,06.09.2023,Курносова О.А.,211- Прочие исковые дела,33-38950/2023,"Отказано в удовлетворении, 07.02.2023","Отменено частично, 06.09.2023",https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
6,77RS0002-02-2022-018700-20,02-0986/2023 (02-54...,"Истец: Никонова Ю.А.Ответчик: ФГАОУ ВО ""НИУ ""В...",14.09.2022,14.02.2023,28.04.2023,Курносова О.А.,"74 - Иные, возникающие из трудовых правоотношений",NaN,"Отказано в удовлетворении, 14.02.2023",NaN,https://mos-gorsud.ru/rs/basmannyj/services/ca...,NaN
7,77RS0006-02-2022-010324-07,02-0681/2023 (02-44...,Истец: Жарова А.К.Ответчик: НИУ ВШЭ,11.08.2022,15.03.2023,16.05.2023,Александренко И.М.,54 - Трудовые споры о взыскании невыплаченной ...,NaN,"Отказано в удовлетворении, 15.03.2023",NaN,https://mos-gorsud.ru/rs/dorogomilovskij/servi...,NaN
8,77RS0004-02-2023-005000-96,02а-0622/2023 ...,"Административный истец: АО ""Московский областн...",18.04.2023,21.06.2023,NaN,Кочнева А.Н.,"27 - Прочие об оспаривании решений, действий (...",NaN,"Отказано в удовлетворении, 21.06.2023",NaN,https://mos-gorsud.ru/rs/gagarinskij/services/...,NaN
9,77RS0015-02-2023-009457-13,02а-0621/2023 ...,Административный истец: Журкин М.Д.Администрат...,06.06.2023,22.06.2023,NaN,Кац Ю.А.,"26 - Об оспаривании решений, действий (бездейс...",33а-2648/2024,"Удовлетворено, 22.06.2023",NaN,https://mos-gorsud.ru/rs/lyublinskij/services/...,NaN


## 4. Selenium: явные и неявные ожидания и обработка исключений<a name='part4'></a>

Как вы могли заметить, некоторые сайты загружаются очень постепенно. Сейчас мы с вами использовали ожидания с помощью `time.sleep(seconds)` чтобы перестраховаться от получения `ElementNotVisibleException` – исключения, которое возникает, когда вы пытаетесь взаимодействовать с элементом который еще не подгрузился на страницу. Как считают авторы документации к Selenium на Python:

> *Худший пример такого кода (явного ожидания) — это использование команды `time.sleep()`, которая устанавливает точное время ожидания.*
---

**Почему плохо использовать `time.sleep()`?** Если сайт, с которым вы работаете, долго грузится, то вы скорее всего для всех страниц будете выставлять достаточно большое значение - например, будете ждать 3-5 секунд. Однако, некоторые страницы могут загрузиться у вас быстрее (например, у вас вырастет скорость интернета). Это не страшно когда вы работаете с учебными проектами, но важно, когда занимаетесь сбором данных по работе (в том числе, от вас ждут данные заказчики или коллеги):

Представим себе следующую ситуацию:
* Вам нужно собрать данные с `10 000` страниц;
* Вы проставили 5 секунд ожидания в `time.sleep()` потому что у вас не подгружались моментально все элементы;
* Первая половина страниц действительно прогружалась по 5 секунд, а вот для второй половины – все искомые элементы загружались за 3.5 секунды.

**Тогда:** вы затратили на сбор данных с 10 тысяч страниц минимум 50 тысяч секунд (833 минуты или 13 часов – собирали все десять тысяч страниц по 5 секунд), **а могли бы затратить** меньше –  как минимум, 42.5 тысячи секунд (708 минут или 11 часов 48 минут - собирали пять тысяч страниц по 5 секунд, другие пять тысяч страниц по 3.5 секунды). 

**Кажется, что разница небольшая**. Но на самом деле это означает, что вам не придется писать заказчику или коллеге `Привет, данные будут через час` или что у вас будет время их посмотреть, написать по ним справку, оформить и дообработать при необходимости. Или вы хотя бы увидите фатальные ошибки на час раньше :)

---

Но есть еще несколько более цивилизованных способов работы с ожиданиями: **неявные** и **явные** ожидания в Selenium.

**`wb.implicitly_wait()` и неявные ожидания**

Что это за неявное ожидание такое? Неявное ожидание указывает WebDriverу опрашивать html определенное количество времени (чаще - секунд), когда пытается найти элемент или элементы. Значение по умолчанию равно 0. Если коротко - это простой путь: вы говорите – *Webdriver, ищи этот элемент столько-то секунд*.

Давайте посмотрим на примере:

In [179]:
link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'

wb.implicitly_wait(3) # будем ждать 3 секунды
wb.get(link_to_one_case) # открываем страницу

open_documents = wb.find_element(By.ID, 'ui-id-3') # ищем кнопку "Документы"
open_documents.click() # кликаем

**`try-except` и работа с исключениями в Selenium**
Код выше был выполнен успешно, потому что у нас с вами действительно есть этот элемент на странице. А теперь давайте попробуем переписать наш код под взаимодействие с исключениями.

Названия всех распространенных исключений в Selenium можно найти по [ссылке](https://www.selenium.dev/selenium/docs/api/py/common/selenium.common.exceptions.html).

In [190]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException

link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'

wb.implicitly_wait(3) # будем ждать 3 секунды
wb.get(link_to_one_case) # открываем страницу

non_existed_id = 'myDinamicElement'

try:
    open_documents = wb.find_element(By.ID, non_existed_id) # ищем кнопку 
    open_documents.click() # кликаем
except ElementNotVisibleException: # работаем с исключениями - если элемент не видим
    print('Вебдрайвер не может увидеть этот элемент')
except NoSuchElementException: # если элемент не найден
    print('Такой элемент не найден')
except TimeoutException: # если время вышло
    print('Время вышло!')

Такой элемент не найден


**`WebDriverWait() и явные ожидания`**

Явное ожидание — это код, которым вы определяете какое необходимое условие должно произойти для того, чтобы дальнейший код исполнился. Список всех явных ожиданий доступен в [документации](https://selenium-python.readthedocs.io/waits.html), но приведу несколько примеров:

* `presence_of_element_located` - ожидание, которое проверяет присутствует ли элемент в html страницы (не обязательно должен быть видимым);
* `visibility_of_element_located` - ожидание, которое проверяет присутствует ли элемент в html страницы и видим ли он. Видимость означает что элемент не только отображается, но также имеет высоту (height) и ширину (width) больше, чем 0. 
* `element_to_be_clickable` – ожидание, которое проверяет, отображается и кликабелен ли элемент.

Какие элементы мы тут используем?
* `WebDriverWait` – класс, отвечающий за ожидание;
* `wait.until()` это метод, отвечающий за ожидание. Внутрь ему мы должны подать то, что и как мы ожидаем.
* `EC.<some value>` – класс, отвечащий за то, что конкретно мы ожидаем. В качестве аргумента подаем локатор элемента `(By.ID, 'ui-id-3')`.

Воспользуемся явными ожиданиями сами:

In [195]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

link_to_one_case = 'https://mos-gorsud.ru/rs/basmannyj/services/cases/criminal/details/071d64e0-bf4c-11ed-bd85-6751552d0bab?year=2023&formType=fullForm'
wb.get(link_to_one_case) # открываем ссылку
wait = WebDriverWait(wb, 3) # ждем до 3х секунд ИЛИ пока не найдем элемент, удовлетворяющий условиям:

try:
    # воспользуемся ожиданием, проверяющим, кликабельный ли элемент
    element = wait.until(EC.element_to_be_clickable((By.ID, 'ui-id-3'))) # пытаемся нажать на кнопку "Документы"
    element.click()
    print('Получилось')
except NoSuchElementException:
    print('Не получилось найти элемент.')
except ElementNotInteractableException:
    print('С этим элементом нельзя взаимодействовать')

## 5. os: менеджмент файлов на вашем компьютере <a name='part5'></a>

Модуль `os` выступает в качестве интерфейса между Python и операционной системой, что позволяет управлять путями к файлам, создавать каталоги, получать информацию о запущенных процессах и переменных окружения и выполнять множество других полезных вещей. Документация os доступна по [ссылке](https://docs.python.org/3/library/os.html). Здесь мы сразу же используем и модуль `glob` - он используется для поиска файлов, удовлетворяющих определенным условиям.

In [214]:
import os 
import glob

In [215]:
your_downloads = '/Users/lika.kapustina/Downloads' # поменяйте это значение

list_of_files = glob.glob(f'{your_downloads}/*') # * означает что мы ищем файлы всех форматов.
latest_file = max(list_of_files, key=os.path.getctime) # используем в качестве ключа время
print(latest_file) # напечатает название последнего по времени файла в ваших загрузках (у меня тут файлов нет)

/Users/lika.kapustina/Downloads/$RECYCLE.BIN


Давайте адаптируем код выше к нашему коду и будем не только скачивать в цикле все файлы, но и сохранять информацию о том, для какого дела файл мы сохранили.

In [218]:
# wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [224]:
downloaded_files = {}

for link in tqdm.tqdm(links):
    wb.get(link) # открываем дело
    wait = WebDriverWait(wb, 3) # подождем 3 секунды ИЛИ пока кнопка "документы" не будет найдена
    
    try:
        # Сценарий 1: на странице есть кнопка "Документы" и нужно кликнуть на неё чтобы увидеть файлы
        # воспользуемся ожиданием, проверяющим, кликабельный ли элемент
        element = wait.until(EC.element_to_be_clickable((By.ID, 'ui-id-3'))) # пытаемся нажать на кнопку "Документы"
        element.click() # кликаем на документы
        html = br.page_source
        soup = BeautifulSoup(html)
        links_for_files = [f"https://mos-gorsud.ru{i.get('href')}" for i in soup.find_all('a') if 'Скачать файл' in i.text] # получаем список элементов, которые можно скачать
    except:
        # Сценарий 2: на странице нет кнопки "Документы", но документы для скачивания доступны на основной странице
        try:
            links_for_files = [f"https://mos-gorsud.ru{i.get('href')}" for i in soup.find_all('a') if 'Скачать файл' in i.text] # получаем список элементов, которые можно скачать
        # Сценарий 3: на странице не найдено
        except:
            print(f'На странице {link} не найдено файлов для скачивания.')
            continue # если не было не найдено никаких файлов, сразу переходим к работе со следующей ссылкой
    
    # Перейдем к работе со следующей ссылкой если ошибок не возникло, но нет ссылок для скачивания
    if len(links_for_files) == 0:
        continue
        
    # Этот блок кода запускается если у нас есть какие-то ссылки для скачивания 
    downloaded_files[link] = [] # создаем новый ключ (ссылка) и присваиваем значение - пустой список.
     
    # Пройдемся по всем ссылкам на скачивание на странице ОДНОГО дела
    for one_file_link in links_for_files:
        wb.get(one_file_link)
        time.sleep(2) # простите мне паузу в 2 секунды, это нужно чтобы файл скачался
        
        # теперь получим название последнего скачанного файла
        list_of_files = glob.glob(f'{your_downloads}/*') 
        latest_file = max(list_of_files, key=os.path.getctime)
        
        downloaded_files[link].append(latest_file)

100%|███████████████████████████████████████████| 30/30 [18:11<00:00, 36.38s/it]


Теперь посмотрим на то что получилось:

In [225]:
downloaded_files

{'https://mos-gorsud.ru/rs/nikulinskij/services/cases/civil/details/16cd8a51-cf91-11ed-901b-a51792e7d9cf?year=2023&formType=fullForm': ['/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (30).docx',
  '/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (31).docx'],
 'https://mos-gorsud.ru/rs/horoshevskij/services/cases/civil/details/63577af0-c944-11ed-ab7e-afa5fd1f1a80?year=2023&formType=fullForm': ['/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (32).docx',
  '/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (33).docx'],
 'https://mos-gorsud.ru/rs/basmannyj/services/cases/civil/details/7da80261-cd6f-11ed-91bc-678868b51ab8?year=2023&formType=fullForm': ['/Users/lika.kapustina/Downloads/Дело 02-4564_2023. Мотивированное решение. документ - обезличенная копия (34).docx

<p></p>
<center><b><font size=4>Задача №4. Менеджмент файлов </font></b></center>

**Воспользуйтесь поиском по документации `os`, stackoverflow и другими ресурсами и сделайте следующее**:
* Создайте папку `mosgorsud_hse_data` на вашем **рабочем столе**;
* Пройдитесь по всем документам из вашего словаря и, используя функции os, **переместите эти файлы в эту папку**;
* Перезапишите в словарь новый путь к файлу;
* Напечатайте словарь.

In [ ]:
# YOUR CODE HERE